In [1]:
import pandas as pd
import os
from tkinter import Tk, filedialog

root = Tk() # pointing root to Tk() to use it as Tk() in program.
root.withdraw() # Hides small tkinter window.
root.attributes('-topmost', True) # Opened windows will be active. above all windows despite of selection.

direct = filedialog.askdirectory() # Returns opened path as str

In [2]:
def listFiles(root): # listdir
    allFiles = []; walk = [root]
    while walk:
        folder = walk.pop(0)+"/"; items = os.listdir(folder) # items = folders + files
        for i in items: i=folder+i; (walk if os.path.isdir(i) else allFiles).append(i)
    return allFiles

filenames = list(filter(lambda x: '_InjectionRate.csv' in x, listFiles(direct)))

In [3]:
summary=[]

PW = [3000,2500,1500,1000,900,800,750,700,650,600,560,530,500,470,440,400,380,350,320,300,280,260,240,220,200]
ran = range(2,len(PW)+2)

pulse=dict(zip(ran,PW))

dfs = {
    pws: pd.DataFrame({'Time' : []}) # create dict with key as number of file and value of empty df
    for pws in ran
}

for filename in filenames:
    filenum=int(filename[-23:-19]) # get number of file (PW)
    
    res_df = (
        pd.read_csv(filename,
        sep='\t', encoding='cp852', skiprows=[1,2]) # create data frame from file
    )
    
    sn = filename[-52:-32] # get SN
    rows=int((pulse[filenum]+1000)*1.3) # number of rows in df
    
    res_df=res_df[['Time','TTL','AI2']].rename(columns={'AI2': sn}).head(rows) # get specific columns
    
    dfs[filenum]=dfs[filenum].merge(res_df, how='outer') # merge df for the same PW

In [4]:
with pd.ExcelWriter(f'{direct}\current.xlsx') as writer: # write to excel all df's
    for pws in ran:
        dfs[pws].to_excel(writer, sheet_name=str(pulse[pws]) , index=False) 